# Step 1: Develop Initial Scraper and Crawler

Scraping HHS OIG Enforcement Actions

In [1]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# Set target URL
url = "https://oig.hhs.gov/fraud/enforcement/"

# Send HTTP request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)
print(f"Status code: {response.status_code}")

Status code: 200


In [3]:
import re
from datetime import datetime

# Parse HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find enforcement action entries
enforcement_actions = []

# Look for h2 headings and use their parent containers
h2_tags = soup.find_all('h2')
articles = [h2.parent for h2 in h2_tags if h2.parent]

print(f"Found {len(articles)} enforcement action entries")


Looking for 'article' tags: 0
Looking for 'h2' tags: 24
Looking for 'h3' tags: 1
Looking for divs with relevant classes: 0

First h2 found: In This Section
Parent of h2: aside, class: ['block-highlight', 'padding-top-2px', 'margin-bottom-4']
Found 24 items based on h2 tags
Total enforcement action entries to process: 24


In [4]:
# Extract information for each enforcement action
for idx, article in enumerate(articles):
    # Extract title and link - try multiple methods
    title = None
    link = None
    
    # Method 1: Look for h2 or h3 with link
    heading = article.find(['h2', 'h3'])
    if heading:
        link_element = heading.find('a')
        if link_element:
            title = link_element.get_text(strip=True)
            link = link_element.get('href')
            # If link is relative path, convert to absolute path
            if link and not link.startswith('http'):
                link = 'https://oig.hhs.gov' + link
        else:
            title = heading.get_text(strip=True)
    
    # Method 2: If no heading, look for any link
    if not title:
        link_element = article.find('a')
        if link_element:
            title = link_element.get_text(strip=True)
            link = link_element.get('href')
            if link and not link.startswith('http'):
                link = 'https://oig.hhs.gov' + link
    
    # Skip if no title found
    if not title:
        continue
    
    # Extract date
    date = None
    # Method 1: Look for time tag
    date_element = article.find('time')
    if date_element:
        date = date_element.get_text(strip=True)
    else:
        # Method 2: Look for date pattern in text
        text = article.get_text()
        date_pattern = r'(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}'
        match = re.search(date_pattern, text)
        if match:
            date = match.group(0)
    
    # Extract category
    categories = []
    category_elements = article.find_all('li')
    for cat in category_elements:
        cat_text = cat.get_text(strip=True)
        if cat_text:
            categories.append(cat_text)
    category = ', '.join(categories) if categories else None
    
    # Add to list
    enforcement_actions.append({
        'title': title,
        'date': date,
        'category': category,
        'link': link
    })

print(f"\nSuccessfully extracted {len(enforcement_actions)} records")

# Show first few records for verification
if enforcement_actions:
    print("\nFirst 3 records:")
    for i, action in enumerate(enforcement_actions[:3]):
        print(f"\n{i+1}. Title: {action['title'][:80]}...")
        print(f"   Date: {action['date']}")
        print(f"   Category: {action['category']}")


Successfully extracted 24 records

First 3 records:

1. Title: In This Section...
   Date: None
   Category: About Enforcement Actions, Civil Monetary Penalty Authorities

2. Title: Archives...
   Date: None
   Category: None

3. Title: Houston Transplant Doctor Indicted For Making False Statements In Patients’ Medi...
   Date: February 5, 2026
   Category: Criminal and Civil Actions


In [5]:
# Create DataFrame
df = pd.DataFrame(enforcement_actions)

print("Top rows in dataset:")
print(df.head())

Top rows in dataset:
                                               title              date  \
0                                    In This Section              None   
1                                           Archives              None   
2  Houston Transplant Doctor Indicted For Making ...  February 5, 2026   
3  MultiCare Health System to Pay Millions to Set...  February 4, 2026   
4  Brooklyn Banker Pleads Guilty to Laundering Pr...  February 3, 2026   

                                            category  \
0  About Enforcement Actions, Civil Monetary Pena...   
1                                               None   
2                         Criminal and Civil Actions   
3                         Criminal and Civil Actions   
4                                           COVID-19   

                                                link  
0                                               None  
1                                               None  
2  https://oig.hhs.gov/fraud/enf

In [6]:
# basic information of the dataframe
print("\nDataset shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nData types:")
print(df.dtypes)


Dataset shape: (24, 4)

Columns: ['title', 'date', 'category', 'link']

Data types:
title       object
date        object
category    object
link        object
dtype: object


In [7]:
# first 10 records
print("\nTop 10 records:")
pd.set_option('display.max_columns', None) # set the maximum number of columns to display to None
pd.set_option('display.max_colwidth', None) # set the maximum width of the columns to None
df.head(10)


Top 10 records:


,title,date,category,link
0,In This Section,None,"About Enforcement Actions, Civil Monetary Penalty Authorities",None
1,Archives,None,None,None
2,Houston Transplant Doctor Indicted For Making False Statements In Patients’ Medical Records,"February 5, 2026",Criminal and Civil Actions,https://oig.hhs.gov/fraud/enforcement/houston-transplant-doctor-indicted-for-making-false-statements-in-patients-medical-records/
3,MultiCare Health System to Pay Millions to Settle Fraud Case,"February 4, 2026",Criminal and Civil Actions,https://oig.hhs.gov/fraud/enforcement/multicare-health-system-to-pay-millions-to-settle-fraud-case/
4,Brooklyn Banker Pleads Guilty to Laundering Proceeds of Medicare Fraud for Transnational Criminal Organization,"February 3, 2026",COVID-19,https://oig.hhs.gov/fraud/enforcement/brooklyn-banker-pleads-guilty-to-laundering-proceeds-of-medicare-fraud-for-transnational-criminal-organization/
5,Delafield Man Sentenced to 18 Months’ Imprisonment for Conspiracy to Pay Health Care Kickbacks,"February 3, 2026",Criminal and Civil Actions,https://oig.hhs.gov/fraud/enforcement/delafield-man-sentenced-to-18-months-imprisonment-for-conspiracy-to-pay-health-care-kickbacks/
6,Former NFL Player Convicted for $197M Medicare Fraud,"February 3, 2026",Criminal and Civil Actions,https://oig.hhs.gov/fraud/enforcement/former-nfl-player-convicted-for-197m-medicare-fraud/
7,Attorney General Hanaway Obtains Medicaid Fraud Conviction Against Couple Lying About Marital Status To Steal Funds,"February 3, 2026",State Enforcement Agencies,https://oig.hhs.gov/fraud/enforcement/attorney-general-hanaway-obtains-medicaid-fraud-conviction-against-couple-lying-about-marital-status-to-steal-funds/
8,"AG's Office Secures Indictments Against Peabody Alcohol and Drug Counselor and Her Businesses for More Than $850,000 in MassHealth Fraud","February 2, 2026",State Enforcement Agencies,https://oig.hhs.gov/fraud/enforcement/ags-office-secures-indictments-against-peabody-alcohol-and-drug-counselor-and-her-businesses-for-more-than-850000-in-masshealth-fraud/
9,Florida Man Pleads Guilty to Conspiracy to Violate the Anti-Kickback Statute,"January 30, 2026",Criminal and Civil Actions,https://oig.hhs.gov/fraud/enforcement/florida-man-pleads-guilty-to-conspiracy-to-violate-the-anti-kickback-statute/


# Step 2: Making the Scraper Dynamic

Creating a Function to Scrape Enforcement Actions by Date Range

### Part a: Pseudo-code for Dynamic Scraper Function

Function: `scrape_enforcement_actions(month, year, run=False)`

1. Input validation:
   - Check if year >= 2013 (data only available from 2013 onwards)
   - If year < 2013, print warning message and return None
   - If run=False, print message and return None without scraping

2. Initialize variables:
   - Create empty list to store all enforcement actions
   - Set base URL for enforcement actions page
   - Set up headers for HTTP requests
   - Calculate target start date from month/year input

3. Loop through pages (using a while loop):
   - Why while loop? We don't know how many pages we need in advance - we need to keep scraping until we reach the target date or run out of pages
   - Start with page 1
   - For each page:
     - Make HTTP request to current page URL
     - Add 1 second delay to avoid server blocking
     - Parse HTML content
     - Extract all enforcement action articles
     - For each article:
       - Extract title, date, category, and link
       - Parse the date and compare with target start date
       - If date >= target date, add to list
       - If date < target date, set flag to stop pagination
     - Check if there's a "Next" button/link for pagination
     - If no more pages or reached target date, exit loop
     - Otherwise, increment page number and continue

4. **Process results:**
   - Convert list to pandas DataFrame
   - Create filename: "enforcement_actions_{year}_{month}.csv"
   - Save DataFrame to CSV file
   - Print summary statistics (total records, date range)
   - Return DataFrame

**Loop type rationale:** 
- While loop is most appropriate because:
  - We don't know the total number of pages beforehand
  - We need to check conditions (date range, page availability) on each iteration
  - We may need to stop early if we reach our target date
  - More flexible than for loop with range()

In [8]:
# Import additional libraries for date handling
from datetime import datetime
import re

# We don't need dateutil for this project,All date parsing can be done with datetime.strptime()

### Part b: Run the Dynamic Scraper

Test the function by scraping enforcement actions since January 2024.

In [10]:
# Test with run=False first (should not execute)
test_df = scrape_enforcement_actions(month=1, year=2024, run=False)

Function not executed. Set run=True to scrape data.


In [11]:
# Now run with run=True to actually scrape data from January 2024
df_2024 = scrape_enforcement_actions(month=1, year=2024, run=True)

Starting to scrape enforcement actions from 1/2024 to today...
Scraping page 1... (URL: https://oig.hhs.gov/fraud/enforcement/)
No more articles found on page 1. Stopping.
No enforcement actions found in the specified date range.


In [12]:
# Analyze the results
if df_2024 is not None:
    print("\n" + "="*60)
    print("ANALYSIS OF SCRAPED DATA")
    print("="*60)
    
    # Total number of enforcement actions
    total_actions = len(df_2024)
    print(f"\n1. Total enforcement actions collected: {total_actions}")
    
    # Display first few rows
    print(f"\n2. First few records:")
    print(df_2024.head())
    
    # Find the earliest enforcement action
    print(f"\n3. Earliest Enforcement Action Details:")
    print("-" * 60)
    
    # Get the last row (assuming data is sorted by date descending)
    earliest_action = df_2024.iloc[-1]
    
    print(f"Date: {earliest_action['date']}")
    print(f"Title: {earliest_action['title']}")
    print(f"Category: {earliest_action['category']}")
    print(f"Link: {earliest_action['link']}")
    
    # Also show the most recent action
    print(f"\n4. Most Recent Enforcement Action Details:")
    print("-" * 60)
    most_recent = df_2024.iloc[0]
    print(f"Date: {most_recent['date']}")
    print(f"Title: {most_recent['title']}")
    print(f"Category: {most_recent['category']}")
    print(f"Link: {most_recent['link']}")
    
    print("\n" + "="*60)

In [13]:
# Display DataFrame info and sample records
if df_2024 is not None:
    print("\nDataFrame Information:")
    print(df_2024.info())
    
    print("\n\nSample of 10 records:")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', 50)
    display(df_2024.head(10))

Check the actual HTML structure

In [14]:
# Check the HTML structure
url = "https://oig.hhs.gov/fraud/enforcement/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)
print(f"Status code: {response.status_code}")
print(f"Response length: {len(response.content)} bytes")

soup = BeautifulSoup(response.content, 'html.parser')

# Check for different possible structures
print("\n--- Checking for 'article' tags ---")
articles = soup.find_all('article')
print(f"Found {len(articles)} article tags")

print("\n--- Checking for other possible containers ---")
# Check for divs with specific classes
divs_with_class = soup.find_all('div', class_=re.compile('item|action|enforcement|card'))
print(f"Found {len(divs_with_class)} divs with relevant classes")

# Check for list items
list_items = soup.find_all('li', class_=re.compile('item|action'))
print(f"Found {len(list_items)} list items with relevant classes")

# Look for h2 or h3 headings (titles)
headings = soup.find_all(['h2', 'h3'])
print(f"Found {len(headings)} h2/h3 headings")

# Show first few headings
if headings:
    print("\nFirst 3 headings:")
    for i, h in enumerate(headings[:3]):
        print(f"{i+1}. {h.get_text(strip=True)[:100]}")
        print(f"   Parent: {h.parent.name}, Parent class: {h.parent.get('class')}")
        print()

Status code: 200
Response length: 73558 bytes

--- Checking for 'article' tags ---
Found 0 article tags

--- Checking for other possible containers ---
Found 20 divs with relevant classes
Found 80 list items with relevant classes
Found 25 h2/h3 headings

First 3 headings:
1. In This Section
   Parent: aside, Parent class: ['block-highlight', 'padding-top-2px', 'margin-bottom-4']

2. Filters
   Parent: form, Parent class: ['filtered-list-form', 'input', 'select', 'option']

3. Archives
   Parent: div, Parent class: ['margin-y-5']



In [ ]:
# look for the main content area and see what structure contains the enforcement actions
print("--- Looking for main content structure ---\n")

# Try to find the main results container
results_div = soup.find('div', class_='views-view-content')
if results_div:
    print("Found 'views-view-content' div")
    children = results_div.find_all(recursive=False)
    print(f"Direct children: {len(children)}")
    if children:
        print(f"First child tag: {children[0].name}, class: {children[0].get('class')}")
        
# Look for specific patterns in the HTML
print("\n--- Checking for table structure ---")
tables = soup.find_all('table')
print(f"Found {len(tables)} tables")

# Check for a views row pattern (common in Drupal sites)
print("\n--- Checking for views/row pattern ---")
rows = soup.find_all('div', class_=re.compile('views-row|row'))
print(f"Found {len(rows)} rows with 'views-row' or 'row' class")

if rows:
    print("\nFirst row structure:")
    first_row = rows[0]
    print(f"Classes: {first_row.get('class')}")
    print(f"Text (first 200 chars): {first_row.get_text(strip=True)[:200]}")
    
    # Check what's inside the row
    title = first_row.find(['h2', 'h3', 'a'])
    if title:
        print(f"\nTitle element: {title.name}, text: {title.get_text(strip=True)[:100]}")

--- Looking for main content structure ---


--- Checking for table structure ---
Found 0 tables

--- Checking for views/row pattern ---
Found 16 rows with 'views-row' or 'row' class

First row structure:
Classes: ['grid-row', 'grid-gap-lg']
Text (first 200 chars): Official websites use .govA.govwebsite belongs to an official government organization in the United States.Secure .gov websites use HTTPSAlock(LockA locked padlock) orhttps://means you’ve safely conne


Updated Scraper Function

In [15]:
def scrape_enforcement_actions(month, year, run=False):
    """
    Improved scraper for HHS OIG enforcement actions from a given month/year to today.
    
    Parameters:
    month : int
        Starting month (1-12)
    year : int
        Starting year (must be >= 2013)
    run : bool
        If False, function will not execute (default: False)
        
    Returns:
    pandas.DataFrame or None
        DataFrame containing scraped enforcement actions, or None if not run
    """
    
    # Check if function should run
    if not run:
        print("Function not executed. Set run=True to scrape data.")
        return None
    
    # Validate year
    if year < 2013:
        print(f"Error: Year must be >= 2013. Only enforcement actions after 2013 are listed.")
        print(f"You entered: {year}")
        return None
    
    # Validate month
    if month < 1 or month > 12:
        print(f"Error: Month must be between 1 and 12. You entered: {month}")
        return None
    
    print(f"Starting to scrape enforcement actions from {month}/{year} to today...")
    
    # Create target start date
    target_date = datetime(year, month, 1)
    
    # Initialize variables
    all_enforcement_actions = []
    base_url = "https://oig.hhs.gov/fraud/enforcement/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    page_num = 0
    should_continue = True
    
    # Loop through pages until we reach target date or run out of pages
    while should_continue:
        # Construct URL for current page
        if page_num == 0:
            url = base_url
        else:
            url = f"{base_url}?page={page_num}"
        
        print(f"Scraping page {page_num + 1}... (URL: {url})")
        
        # Make HTTP request
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {page_num + 1}: {e}")
            break
        
        # Parse HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Try multiple strategies to find enforcement actions
        items = []
        
        # Strategy 1: Look for article tags
        items = soup.find_all('article')
        
        # Strategy 2: Look for views-row divs (common in Drupal)
        if not items:
            items = soup.find_all('div', class_=re.compile('views-row'))
        
        # Strategy 3: Look for specific list items
        if not items:
            items = soup.find_all('li', class_=re.compile('enforcement|action'))
        
        # Strategy 4: Look for divs with h2 headings
        if not items:
            main_content = soup.find('div', class_='views-view-content')
            if main_content:
                items = main_content.find_all('div', recursive=False)
        
        if not items:
            print(f"No items found on page {page_num + 1}. Stopping.")
            break
        
        print(f"  Found {len(items)} items on this page")
        
        # Track if we found any articles within date range on this page
        found_in_range = False
        
        # Extract information from each item
        for item in items:
            # Extract title and link - try multiple methods
            title = None
            link = None
            
            # Look for h2 or h3 with a link
            heading = item.find(['h2', 'h3'])
            if heading:
                link_element = heading.find('a')
                if link_element:
                    title = link_element.get_text(strip=True)
                    link = link_element.get('href')
                    if link and not link.startswith('http'):
                        link = 'https://oig.hhs.gov' + link
                else:
                    title = heading.get_text(strip=True)
            
            # If no heading found, look for any link
            if not title:
                link_element = item.find('a')
                if link_element:
                    title = link_element.get_text(strip=True)
                    link = link_element.get('href')
                    if link and not link.startswith('http'):
                        link = 'https://oig.hhs.gov' + link
            
            # Skip if no title found
            if not title:
                continue
            
            # Extract date - try multiple methods
            date_str = None
            article_date = None
            
            # Method 1: Look for time tag
            date_element = item.find('time')
            if date_element:
                date_str = date_element.get_text(strip=True)
            
            # Method 2: Look for date in specific div/span
            if not date_str:
                date_div = item.find(['div', 'span'], class_=re.compile('date|time'))
                if date_div:
                    date_str = date_div.get_text(strip=True)
            
            # Method 3: Look for date pattern in text
            if not date_str:
                text = item.get_text()
                # Look for common date patterns
                date_pattern = r'(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}'
                match = re.search(date_pattern, text)
                if match:
                    date_str = match.group(0)
            
            # Parse date if found
            if date_str:
                try:
                    article_date = datetime.strptime(date_str, "%B %d, %Y")
                except:
                    try:
                        # Try another format
                        date_str_clean = re.sub(r'\s+', ' ', date_str.strip())
                        article_date = datetime.strptime(date_str_clean, "%B %d, %Y")
                    except:
                        # If parsing fails, use current date to include the article
                        article_date = datetime.now()
            else:
                # If no date found, use current date to include the article
                article_date = datetime.now()
            
            # Check if article is within our date range
            if article_date >= target_date:
                found_in_range = True
                
                # Extract categories
                categories = []
                # Look for list items that might be categories
                category_elements = item.find_all('li')
                for cat in category_elements:
                    cat_text = cat.get_text(strip=True)
                    if cat_text and len(cat_text) > 0:
                        categories.append(cat_text)
                
                # If no categories in list items, look for spans or divs with category class
                if not categories:
                    cat_elements = item.find_all(['span', 'div'], class_=re.compile('category|type|tag'))
                    for cat in cat_elements:
                        cat_text = cat.get_text(strip=True)
                        if cat_text:
                            categories.append(cat_text)
                
                category = ', '.join(categories) if categories else None
                
                # Add to list
                all_enforcement_actions.append({
                    'title': title,
                    'date': date_str,
                    'category': category,
                    'link': link
                })
            else:
                # If we've gone past our target date, we can stop
                print(f"  Reached articles before target date ({date_str}). Stopping.")
                should_continue = False
                break
        
        # If we didn't find anything in range on this page, continue to next page
        # Check for next page
        pagination = soup.find(['nav', 'div'], class_=re.compile('pag'))
        has_next = False
        
        if pagination:
            # Look for next link
            next_link = pagination.find('a', text=re.compile(r'Next|›|»|next', re.IGNORECASE))
            if next_link and 'disabled' not in next_link.get('class', []):
                has_next = True
            
            # Also check for numbered page links
            if not has_next:
                page_links = pagination.find_all('a', href=re.compile(r'\?page=\d+'))
                if page_links:
                    has_next = True
        
        if not has_next:
            print("  No next page available. Stopping.")
            should_continue = False
        
        # Increment page number
        page_num += 1
        
        # Safety: don't scrape more than 100 pages
        if page_num >= 100:
            print("  Reached maximum page limit (100). Stopping.")
            should_continue = False
        
        # Add delay to avoid overwhelming the server
        if should_continue:
            time.sleep(1)
    
    # Create DataFrame
    if all_enforcement_actions:
        df = pd.DataFrame(all_enforcement_actions)
        
        # Save to CSV
        filename = f"enforcement_actions_{year}_{month}.csv"
        df.to_csv(filename, index=False)
        
        print(f"\n{'='*60}")
        print(f"Scraping complete!")
        print(f"Total enforcement actions collected: {len(df)}")
        if 'date' in df.columns and not df['date'].isna().all():
            print(f"Date range: {df['date'].iloc[-1]} to {df['date'].iloc[0]}")
        print(f"Saved to: {filename}")
        print(f"{'='*60}")
        
        return df
    else:
        print("No enforcement actions found in the specified date range.")
        return None

In [16]:
# Display results
# Check if variable exists first
try:
    if df_2024_v2 is not None and len(df_2024_v2) > 0:
        print("SUCCESS! Data scraped successfully.\n")
        print(f"Total records: {len(df_2024_v2)}")
        print(f"\nFirst 5 records:")
        display(df_2024_v2.head())
        
        print(f"\nEarliest enforcement action:")
        earliest = df_2024_v2.iloc[-1]
        print(f"Date: {earliest['date']}")
        print(f"Title: {earliest['title']}")
        print(f"Category: {earliest['category']}")
        print(f"Link: {earliest['link']}")
    else:
        print("Still no data. Let's check the HTML structure manually.")
except NameError:
    print("⚠️ Error: df_2024_v2 is not defined.")
    print("This means the scraper in the previous cell didn't create the variable.")
    print("\nPossible reasons:")
    print("1. The scraper function returned None (no data found)")
    print("2. The previous cell wasn't executed")
    print("3. The scraper encountered an error")
    print("\nPlease run the previous cell first, or check the scraper output for errors.")

NameError: name 'df_2024_v2' is not defined

Test the Scraper on First Page Before Running Full Scrape

### 📊 Interpreting the Test Results

**What to look for:**

1. **If you see "Found: 0" for all strategies** → The website might have changed its structure or is blocking scraping
   - Solution: Try saving the HTML and inspecting it manually
   
2. **If one strategy shows > 0 items** → We can use that strategy!
   - Look for which strategy worked (has a non-zero count)
   - We'll update the scraper to use that specific method
   
3. **If manual extraction succeeds** → The data is there, we just need to adjust our selectors
   - Note which elements were found (✅) and which weren't (❌)
   - We'll update the scraper based on this information

**Next steps after running the test above:**
- Share the output with me so I can see which strategy works
- I'll then fix the scraper function to use the correct HTML structure

If still having issues, save the HTML for manual inspection Uncomment to save HTML

with open('enforcement_page.html', 'w', encoding='utf-8') as f:
f.write(response.text)
print("HTML saved to enforcement_page.html for manual inspection")

### Part c: Collect Enforcement Actions Since January 2022

Now let's scrape all enforcement actions from January 2022 to today. This will take a while as it needs to go through multiple pages.

In [36]:
# Scrape enforcement actions from January 2022 to today
# This will take several minutes as it needs to process multiple pages
df_2022 = scrape_enforcement_actions(month=1, year=2022, run=True)

Starting to scrape enforcement actions from 1/2022 to today...
Scraping page 1... (URL: https://oig.hhs.gov/fraud/enforcement/)
No items found on page 1. Stopping.
No enforcement actions found in the specified date range.


In [37]:
# Analyze the results from January 2022 scraping
if df_2022 is not None and len(df_2022) > 0:
    print("\n" + "="*70)
    print("ANALYSIS OF ENFORCEMENT ACTIONS SINCE JANUARY 2022")
    print("="*70)
    
    # Total number of enforcement actions
    total_actions = len(df_2022)
    print(f"\n1. Total enforcement actions collected: {total_actions}")
    
    # Find the earliest enforcement action
    print(f"\n2. Earliest Enforcement Action Details:")
    print("-" * 70)
    
    # Get the last row (data should be in reverse chronological order)
    earliest_action = df_2022.iloc[-1]
    
    print(f"   Date: {earliest_action['date']}")
    print(f"   Title: {earliest_action['title']}")
    print(f"   Category: {earliest_action['category']}")
    print(f"   Link: {earliest_action['link']}")
    
    # Show the most recent action
    print(f"\n3. Most Recent Enforcement Action Details:")
    print("-" * 70)
    most_recent = df_2022.iloc[0]
    print(f"   Date: {most_recent['date']}")
    print(f"   Title: {most_recent['title']}")
    print(f"   Category: {most_recent['category']}")
    print(f"   Link: {most_recent['link']}")
    
    # Show data summary
    print(f"\n4. Data Summary:")
    print("-" * 70)
    print(df_2022.info())
    
    print("\n" + "="*70)
else:
    print("No data collected. Please check the scraping function.")

No data collected. Please check the scraping function.


# Step 3: Plot Data Based on Scraped Data

For this section, we'll use the CSV file created in Step 2 (enforcement_actions_2022_1.csv) to create visualizations.

In [38]:
# Import visualization libraries
import altair as alt
import numpy as np

# Load the data from CSV (if not already in memory)
try:
    df = pd.read_csv('enforcement_actions_2022_1.csv')
    print(f"Loaded {len(df)} records from CSV file")
except FileNotFoundError:
    # If CSV doesn't exist, use the dataframe from memory
    if df_2022 is not None:
        df = df_2022.copy()
        print(f"Using {len(df)} records from memory")
    else:
        print("Error: No data available. Please run the scraper first.")
        df = None

if df is not None:
    print("\nFirst few rows:")
    display(df.head())

Error: No data available. Please run the scraper first.


In [17]:
# Data preprocessing: Parse dates and create month-year column
if df is not None:
    # Parse date column to datetime
    df['date_parsed'] = pd.to_datetime(df['date'], format='%B %d, %Y', errors='coerce')
    
    # Create month-year column for aggregation
    df['month_year'] = df['date_parsed'].dt.to_period('M')
    
    # Sort by date
    df = df.sort_values('date_parsed')
    
    print("Data preprocessing complete!")
    print(f"\nDate range: {df['date_parsed'].min()} to {df['date_parsed'].max()}")
    print(f"\nSample of processed data:")
    display(df[['date', 'date_parsed', 'month_year', 'title']].head())

Data preprocessing complete!

Date range: 2026-01-27 00:00:00 to 2026-02-05 00:00:00

Sample of processed data:


,date,date_parsed,month_year,title
21,"January 27, 2026",2026-01-27,2026-01,"Cordell Memorial Hospital Agreed to Pay $40,000 for Allegedly Violating Patient Dumping Statute by Failing to Provide an Appropriate Medical Screening Examination"
19,"January 28, 2026",2026-01-28,2026-01,Rheumatologist Agrees To Resolve False Claims Act Allegations Related To Unapproved Drugs
18,"January 28, 2026",2026-01-28,2026-01,"Scranton Heart Institute Agrees To Pay $48,709.20 To Settle False Claims Act Allegations"
17,"January 28, 2026",2026-01-28,2026-01,Repeat Health Care Fraud Offender Sentenced for Defrauding New Hampshire Medicaid
16,"January 28, 2026",2026-01-28,2026-01,Slidell Chiropractor Sentenced for Health Care Fraud


## Plot 1: Number of Enforcement Actions Over Time

Line chart showing the number of enforcement actions aggregated by month+year overall since January 2022.

In [20]:
# Import visualization library
try:
    import altair as alt
    USE_ALTAIR = True
except ImportError:
    import matplotlib.pyplot as plt
    USE_ALTAIR = False
    print("Using matplotlib instead of altair")

In [22]:
# Aggregate data by month-year
if df is not None:
    # Count enforcement actions by month
    monthly_counts = df.groupby('month_year').size().reset_index(name='count')
    
    # Convert period to string for Altair
    monthly_counts['month_year_str'] = monthly_counts['month_year'].astype(str)
    
    print("Monthly aggregation:")
    display(monthly_counts.head(10))
    
    # Create line chart with Altair
    chart1 = alt.Chart(monthly_counts).mark_line(point=True).encode(
        x=alt.X('month_year_str:T', 
                title='Month-Year',
                axis=alt.Axis(labelAngle=-45)),
        y=alt.Y('count:Q', 
                title='Number of Enforcement Actions'),
        tooltip=['month_year_str:T', 'count:Q']
    ).properties(
        title='Number of Enforcement Actions Over Time (January 2022 - Present)',
        width=700,
        height=400
    ).configure_axis(      
        labelFontSize=12,
        titleFontSize=14
    ).configure_title(
        fontSize=16
    )
    
    chart1

Monthly aggregation:


,month_year,count,month_year_str
0,2026-01,13,2026-01
1,2026-02,7,2026-02


## Plot 2: Enforcement Actions by Category

create categories based on the enforcement action types:
1. "Criminal and Civil Actions" vs. "State Enforcement Agencies"
2. Within "Criminal and Civil Actions", classify into 5 topics based on title keywords

In [23]:
# Classify enforcement actions by main category
if df is not None:
    # First, determine if it's "Criminal and Civil Actions" or "State Enforcement Agencies"
    df['main_category'] = df['category'].apply(
        lambda x: 'State Enforcement Agencies' if pd.notna(x) and 'State Enforcement' in str(x) 
        else 'Criminal and Civil Actions'
    )
    
    print("Main category distribution:")
    print(df['main_category'].value_counts())
    print("\nSample of categorized data:")
    display(df[['title', 'category', 'main_category']].head(10))

Main category distribution:
main_category
Criminal and Civil Actions    18
State Enforcement Agencies     6
Name: count, dtype: int64

Sample of categorized data:


,title,category,main_category
21,"Cordell Memorial Hospital Agreed to Pay $40,000 for Allegedly Violating Patient Dumping Statute by Failing to Provide an Appropriate Medical Screening Examination","CMP and Affirmative Exclusions, EMTALA/Patient Dumping",Criminal and Civil Actions
19,Rheumatologist Agrees To Resolve False Claims Act Allegations Related To Unapproved Drugs,Criminal and Civil Actions,Criminal and Civil Actions
18,"Scranton Heart Institute Agrees To Pay $48,709.20 To Settle False Claims Act Allegations",Criminal and Civil Actions,Criminal and Civil Actions
17,Repeat Health Care Fraud Offender Sentenced for Defrauding New Hampshire Medicaid,Criminal and Civil Actions,Criminal and Civil Actions
16,Slidell Chiropractor Sentenced for Health Care Fraud,"COVID-19, Criminal and Civil Actions",Criminal and Civil Actions
15,"Holmes Regional Medical Center Agreed to Pay $113,000 for Allegedly Violating Patient Dumping Statute by Failing to Provide an Appropriate Medical Screening Examination","CMP and Affirmative Exclusions, EMTALA/Patient Dumping",Criminal and Civil Actions
20,Attorney General James Uthmeier Announces Arrests in Central Florida Medicaid Fraud Scheme,State Enforcement Agencies,State Enforcement Agencies
14,Attorney General Hanaway Obtains Medicaid Fraud Conviction For Services Not Provided,State Enforcement Agencies,State Enforcement Agencies
13,Attorney General Labrador Announces Sentencing of Kootenai County Woman for Public Assistance Provider Fraud,State Enforcement Agencies,State Enforcement Agencies
12,Yadkinville Woman Sentenced in Connection with Multi-Million Dollar Medicaid Fraud Scheme,Criminal and Civil Actions,Criminal and Civil Actions


In [24]:
# Function to classify Criminal and Civil Actions into 5 topics based on title
def classify_criminal_civil_topic(title):
    """
    Classify enforcement actions into 5 topics based on keywords in the title.
    Topics: Health Care Fraud, Financial Fraud, Drug Enforcement, Bribery/Corruption, Other
    """
    if pd.isna(title):
        return 'Other'
    
    title_lower = title.lower()
    
    # Health Care Fraud keywords
    health_keywords = ['health care', 'healthcare', 'medicare', 'medicaid', 'hospital', 
                       'doctor', 'physician', 'medical', 'clinic', 'prescription',
                       'patient', 'nursing', 'pharmacy', 'drug test', 'diagnostic',
                       'therapy', 'treatment', 'insurance claim', 'billing']
    
    # Financial Fraud keywords
    financial_keywords = ['bank', 'financial', 'money laundering', 'wire fraud', 
                         'securities', 'investment', 'ponzi', 'embezzle', 
                         'tax fraud', 'loan fraud', 'credit card', 'mortgage']
    
    # Drug Enforcement keywords
    drug_keywords = ['drug', 'opioid', 'fentanyl', 'controlled substance', 
                    'narcotic', 'cocaine', 'heroin', 'methamphetamine',
                    'prescription drug', 'illegal distribution', 'dea']
    
    # Bribery/Corruption keywords
    bribery_keywords = ['bribery', 'bribe', 'kickback', 'corruption', 'corrupt',
                       'conspiracy', 'illegal payment', 'unlawful payment']
    
    # Check keywords in order of specificity
    # Check bribery first (often mentioned in titles)
    if any(keyword in title_lower for keyword in bribery_keywords):
        return 'Bribery/Corruption'
    
    # Check drug enforcement
    if any(keyword in title_lower for keyword in drug_keywords):
        return 'Drug Enforcement'
    
    # Check financial fraud
    if any(keyword in title_lower for keyword in financial_keywords):
        return 'Financial Fraud'
    
    # Check health care fraud (most common, check last to avoid over-classification)
    if any(keyword in title_lower for keyword in health_keywords):
        return 'Health Care Fraud'
    
    # Default to Other
    return 'Other'

# Apply classification to Criminal and Civil Actions only
if df is not None:
    df['topic'] = df.apply(
        lambda row: classify_criminal_civil_topic(row['title']) 
        if row['main_category'] == 'Criminal and Civil Actions' 
        else row['main_category'],
        axis=1
    )
    
    print("Topic distribution within Criminal and Civil Actions:")
    criminal_civil = df[df['main_category'] == 'Criminal and Civil Actions']
    print(criminal_civil['topic'].value_counts())
    
    print("\n\nOverall topic distribution:")
    print(df['topic'].value_counts())
    
    print("\n\nSample classifications:")
    display(df[['title', 'main_category', 'topic']].head(20))

Topic distribution within Criminal and Civil Actions:
topic
Health Care Fraud     7
Other                 7
Bribery/Corruption    2
Drug Enforcement      1
Financial Fraud       1
Name: count, dtype: int64


Overall topic distribution:
topic
Health Care Fraud             7
Other                         7
State Enforcement Agencies    6
Bribery/Corruption            2
Drug Enforcement              1
Financial Fraud               1
Name: count, dtype: int64


Sample classifications:


,title,main_category,topic
21,"Cordell Memorial Hospital Agreed to Pay $40,000 for Allegedly Violating Patient Dumping Statute by Failing to Provide an Appropriate Medical Screening Examination",Criminal and Civil Actions,Health Care Fraud
19,Rheumatologist Agrees To Resolve False Claims Act Allegations Related To Unapproved Drugs,Criminal and Civil Actions,Drug Enforcement
18,"Scranton Heart Institute Agrees To Pay $48,709.20 To Settle False Claims Act Allegations",Criminal and Civil Actions,Other
17,Repeat Health Care Fraud Offender Sentenced for Defrauding New Hampshire Medicaid,Criminal and Civil Actions,Health Care Fraud
16,Slidell Chiropractor Sentenced for Health Care Fraud,Criminal and Civil Actions,Health Care Fraud
15,"Holmes Regional Medical Center Agreed to Pay $113,000 for Allegedly Violating Patient Dumping Statute by Failing to Provide an Appropriate Medical Screening Examination",Criminal and Civil Actions,Health Care Fraud
20,Attorney General James Uthmeier Announces Arrests in Central Florida Medicaid Fraud Scheme,State Enforcement Agencies,State Enforcement Agencies
14,Attorney General Hanaway Obtains Medicaid Fraud Conviction For Services Not Provided,State Enforcement Agencies,State Enforcement Agencies
13,Attorney General Labrador Announces Sentencing of Kootenai County Woman for Public Assistance Provider Fraud,State Enforcement Agencies,State Enforcement Agencies
12,Yadkinville Woman Sentenced in Connection with Multi-Million Dollar Medicaid Fraud Scheme,Criminal and Civil Actions,Health Care Fraud


### Plot 2a: Criminal and Civil Actions vs. State Enforcement Agencies Over Time

In [25]:
# Aggregate by month and main category
if df is not None:
    monthly_category = df.groupby(['month_year', 'main_category']).size().reset_index(name='count')
    monthly_category['month_year_str'] = monthly_category['month_year'].astype(str)
    
    print("Monthly counts by main category:")
    display(monthly_category.head(10))
    
    # Create line chart
    chart2a = alt.Chart(monthly_category).mark_line(point=True).encode(
        x=alt.X('month_year_str:T', 
                title='Month-Year',
                axis=alt.Axis(labelAngle=-45)),
        y=alt.Y('count:Q', 
                title='Number of Enforcement Actions'),
        color=alt.Color('main_category:N', 
                       title='Category',
                       scale=alt.Scale(scheme='category10')),
        tooltip=['month_year_str:T', 'main_category:N', 'count:Q']
    ).properties(
        title='Enforcement Actions: Criminal and Civil vs. State Agencies',
        width=700,
        height=400
    ).configure_axis(
        labelFontSize=12,
        titleFontSize=14
    ).configure_title(
        fontSize=16
    )
    
    chart2a

Monthly counts by main category:


,month_year,main_category,count,month_year_str
0,2026-01,Criminal and Civil Actions,9,2026-01
1,2026-01,State Enforcement Agencies,4,2026-01
2,2026-02,Criminal and Civil Actions,5,2026-02
3,2026-02,State Enforcement Agencies,2,2026-02


### Plot 2b: Five Topics within Criminal and Civil Actions Over Time

In [26]:
# Filter for Criminal and Civil Actions only and aggregate by topic
if df is not None:
    criminal_civil_df = df[df['main_category'] == 'Criminal and Civil Actions'].copy()
    
    monthly_topic = criminal_civil_df.groupby(['month_year', 'topic']).size().reset_index(name='count')
    monthly_topic['month_year_str'] = monthly_topic['month_year'].astype(str)
    
    print("Monthly counts by topic (Criminal and Civil Actions only):")
    display(monthly_topic.head(15))
    
    # Create line chart
    chart2b = alt.Chart(monthly_topic).mark_line(point=True).encode(
        x=alt.X('month_year_str:T', 
                title='Month-Year',
                axis=alt.Axis(labelAngle=-45)),
        y=alt.Y('count:Q', 
                title='Number of Enforcement Actions'),
        color=alt.Color('topic:N', 
                       title='Topic',
                       scale=alt.Scale(scheme='category10')),
        tooltip=['month_year_str:T', 'topic:N', 'count:Q']
    ).properties(
        title='Criminal and Civil Actions by Topic Over Time',
        width=700,
        height=400
    ).configure_axis(
        labelFontSize=12,
        titleFontSize=14
    ).configure_title(
        fontSize=16
    ).configure_legend(
        labelFontSize=11
    )
    
    chart2b

Monthly counts by topic (Criminal and Civil Actions only):


,month_year,topic,count,month_year_str
0,2026-01,Bribery/Corruption,1,2026-01
1,2026-01,Drug Enforcement,1,2026-01
2,2026-01,Health Care Fraud,5,2026-01
3,2026-01,Other,2,2026-01
4,2026-02,Bribery/Corruption,1,2026-02
5,2026-02,Financial Fraud,1,2026-02
6,2026-02,Health Care Fraud,2,2026-02
7,2026-02,Other,1,2026-02


In [29]:
# Summary statistics
if df is not None:
    print(f"\n1. Total Enforcement Actions: {len(df)}")
    
    print(f"\n2. Date Range:")
    print(f"   Earliest: {df['date_parsed'].min()}")
    print(f"   Latest: {df['date_parsed'].max()}")
    
    print(f"\n3. Main Category Distribution:")
    print(df['main_category'].value_counts())
    
    print(f"\n4. Topic Distribution (Criminal and Civil Actions):")
    criminal_civil = df[df['main_category'] == 'Criminal and Civil Actions']
    print(criminal_civil['topic'].value_counts())
    
    print(f"\n5. Average Enforcement Actions per Month:")
    avg_per_month = df.groupby('month_year').size().mean()
    print(f"   {avg_per_month:.1f} actions/month")
    
    print(f"\n6. Month with Most Enforcement Actions:")
    max_month = df.groupby('month_year').size().idxmax()
    max_count = df.groupby('month_year').size().max()
    print(f"   {max_month}: {max_count} actions")
    
    print(f"\n7. Month with Least Enforcement Actions:")
    min_month = df.groupby('month_year').size().idxmin()
    min_count = df.groupby('month_year').size().min()
    print(f"   {min_month}: {min_count} actions")


1. Total Enforcement Actions: 24

2. Date Range:
   Earliest: 2026-01-27 00:00:00
   Latest: 2026-02-05 00:00:00

3. Main Category Distribution:
main_category
Criminal and Civil Actions    18
State Enforcement Agencies     6
Name: count, dtype: int64

4. Topic Distribution (Criminal and Civil Actions):
topic
Health Care Fraud     7
Other                 7
Bribery/Corruption    2
Drug Enforcement      1
Financial Fraud       1
Name: count, dtype: int64

5. Average Enforcement Actions per Month:
   10.0 actions/month

6. Month with Most Enforcement Actions:
   2026-01: 13 actions

7. Month with Least Enforcement Actions:
   2026-02: 7 actions


## Explanation of Classification Methodology

### How the Topic Classification Works:

The classification system uses keyword matching to categorize enforcement actions into 5 topics:

1. **Health Care Fraud**: Keywords include "health care", "medicare", "medicaid", "hospital", "doctor", "medical", etc.
   - Example: "Hospital Executive Pleads Guilty to Medicare Fraud Scheme"

2. **Financial Fraud**: Keywords include "bank", "financial", "money laundering", "wire fraud", "securities", etc.
   - Example: "Brooklyn Banker Pleads Guilty to Laundering Proceeds"

3. **Drug Enforcement**: Keywords include "drug", "opioid", "fentanyl", "controlled substance", "narcotic", etc.
   - Example: "Pharmacy Owner Charged with Illegal Distribution of Opioids"

4. **Bribery/Corruption**: Keywords include "bribery", "kickback", "corruption", "conspiracy", etc.
   - Example: "Official Sentenced for Accepting Kickbacks"

5. **Other**: Any enforcement action that doesn't match the above categories
   - Example: General fraud cases not fitting other categories

The classification prioritizes specificity, checking for bribery/corruption and drug enforcement first (as these are often explicitly mentioned), then financial fraud, and finally health care fraud (which is the most common category).